# Setup

In [ ]:
%pip install anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.2/388.2 kB 7.4 MB/s eta 0:00:00


In [ ]:
from anthropic import Anthropic

In [ ]:

client = Anthropic(api_key=API_KEY)

# Workflow blocks

## Utils

In [ ]:
# One shot query to claude
def llm_call(prompt: str, system_prompt: str="", client: Anthropic|None=None, model="claude-3-haiku-20240307"):
  if not client:
    client = Anthropic(api_key=API_KEY)

  messages = [{"role": "user", "content": prompt}]
  response = client.messages.create(
      model=model,
      max_tokens=1024,
      messages=messages,
      system=system_prompt
  )
  return next((block.text for block in response.content if block.type == "text"), "Error: no response from Claude")

In [ ]:
import re

def extract_xml(text: str, tag: str) -> str:
  """Extracts content inside an XML tag from a string."""
  match = re.search(rf"<{tag}>(.*?)</{tag}>", text, re.DOTALL)
  return match.group(1).strip() if match else ""

In [ ]:
llm_call("hello")

'Hello! How can I assist you today?'

In [ ]:
s = """
  Analyze the input and select the best route to take.
  The available routes are:
  <routes>
    a
    b
    c
  </routes>

  Your response must be in the following format:
  <reason>
  Brief explanation of why you choose this route option
  </reason>
  <route>
  The name of the route to take
  </route>
  """
extract_xml(s, "reason")

'Brief explanation of why you choose this route option'

## Basic blocks

### prompt chaining


In [ ]:
def chain_prompt(input: str, prompts: list[str], client: Anthropic|None):
  result = input
  for i, prompt in enumerate(prompts):
    query = f"{prompt}\nInput:{result}"
    result = llm_call(prompt=f"{prompt}\nInput:{result}", client=client)
    print(f"Step {i}")
    print(f"Query: {query}")
    print(f"Result: {result}")
    print("-" * 20)
  return result

In [ ]:
input_str = "How are you today"
prompts = [
    "Rewrite the input with flowery language. Only return the rewritten string. ",
    "Return the input string followed by the Chinese translation in parenthesis",
    "Return the input string in uppercase"
]
chain_prompt(input_str, prompts, client)

Step 0
Query: Rewrite the input with flowery language. Only return the rewritten string. 
Input:How are you today
Result: Greetings, my esteemed companion! I trust this day has bestowed upon you a most magnificent countenance, and that your soul is imbued with the radiant essence of profound contentment. Pray, reveal to me the nature of your current state of being, that I may bask in the splendor of your illuminated spirit.
--------------------
Step 1
Query: Return the input string followed by the Chinese translation in parenthesis
Input:Greetings, my esteemed companion! I trust this day has bestowed upon you a most magnificent countenance, and that your soul is imbued with the radiant essence of profound contentment. Pray, reveal to me the nature of your current state of being, that I may bask in the splendor of your illuminated spirit.
Result: Greetings, my esteemed companion! I trust this day has bestowed upon you a most magnificent countenance, and that your soul is imbued with the

'GREETINGS, MY ESTEEMED COMPANION! I TRUST THIS DAY HAS BESTOWED UPON YOU A MOST MAGNIFICENT COUNTENANCE, AND THAT YOUR SOUL IS IMBUED WITH THE RADIANT ESSENCE OF PROFOUND CONTENTMENT. PRAY, REVEAL TO ME THE NATURE OF YOUR CURRENT STATE OF BEING, THAT I MAY BASK IN THE SPLENDOR OF YOUR ILLUMINATED SPIRIT. (问候,我亲爱的同伴!我相信今天已经给您带来了最出众的外表,并且您的灵魂已经被深深的满足感所充满。请告诉我您当前的状态,让我沐浴在您光辉灿烂的精神中。)'

In [ ]:
input_str = "Google Home Smart Hub"
prompts = [
    "Write a marketing copy of the input product",
    "Translate the input string into French (fr) and Chinese (zh-CN). Return a json where the key is the language code and the value is the translated text",
]
chain_prompt(input_str, prompts, client)

Step 0
Query: Write a marketing copy of the input product
Input:Google Home Smart Hub
Result: Introducing the Google Home Smart Hub - Your Intelligent Home Companion

Elevate your home experience with the power of Google Home Smart Hub. This cutting-edge device seamlessly blends cutting-edge technology with sleek design, transforming your living space into a smart, responsive environment.

Driven by the intelligence of the Google Assistant, the Smart Hub puts a world of information, entertainment, and control at your fingertips. With just a simple voice command, you can access a vast array of features - play music, check the weather, control smart home devices, manage your schedule, and so much more.

The vibrant, high-resolution display offers a stunning visual interface, allowing you to effortlessly view information, watch videos, and even video call loved ones. The device's versatile design makes it a perfect fit for any room, blending seamlessly with your home's décor.

Experience 

'Here is the translation of the input string into French (fr) and Chinese (zh-CN), returned as a JSON:\n\n{\n  "fr": "Découvrez le Google Home Smart Hub - Votre compagnon intelligent de la maison\n\nÉlevez l\'expérience de votre maison avec la puissance du Google Home Smart Hub. Cet appareil de pointe allie parfaitement la technologie de pointe à un design élégant, transformant votre espace de vie en un environnement intelligent et réactif.\n\nPropulsé par l\'intelligence de l\'assistant Google, le Smart Hub vous donne accès à un monde d\'informations, de divertissement et de contrôle au bout des doigts. Avec une simple commande vocale, vous pouvez accéder à une vaste gamme de fonctionnalités - écouter de la musique, vérifier la météo, contrôler les appareils domotiques, gérer votre emploi du temps et bien plus encore.\n\nL\'écran vibrant et haute résolution offre une interface visuelle époustouflante, vous permettant de consulter facilement des informations, de regarder des vidéos et 

### Routing

In [ ]:
from collections import namedtuple

Route = namedtuple("Route", ["name", "desc", "prompt"])

In [ ]:
def route(input: str, routes: list[Route], client: Anthropic|None=None):
  formatted_routes = "\n".join([f"{route.name}: {route.desc}" for route in routes])

  routing_prompt=f"""
  Analyze the input and select the best route to take.
  The available routes are:
  <routes>
    {formatted_routes}
  </routes>

  Your response must be in the following format:
  <reason>
  Brief explanation of why you choose this route option
  </reason>
  <route>
  The name of the route to take
  </route>
  """

  query = f"{routing_prompt}\nInput: {input}"
  print(f"Sending query: {query}")
  result = llm_call(prompt=query, client=client)

  reason = extract_xml(result, "reason")
  route_name = extract_xml(result, "route")
  print(f"Route selected: {route_name}")
  print(f"Reason: {reason}")

  route = next((r for r in routes if r.name == route_name), None)
  if not route:
    # TODO
    raise ValueError(f"Route {route_name} not found")

  query = f"{route.prompt}\nInput: {input}"
  print(f"Sending query: {query}")
  result = llm_call(prompt=query, client=client)
  return result

In [ ]:
routes = [
    Route(
        name="language",
        desc="Handles language/linguistic related question",
        prompt="You are a professional linguist. Answer the input use linguistic knowledge and give a brief explanation. "
    ),
    Route(
        name="math",
        desc="Handles math related question",
        prompt="You are a professional math tutor. Solve the input math problem and show your work step by step. "
    )
]

In [ ]:
route("How to calculate C(5,3)?", routes)

Sending query: 
  Analyze the input and select the best route to take. 
  The available routes are:
  <routes>
    language: Handles language/linguistic related question
math: Handles math related question
  </routes>
  
  Your response must be in the following format:
  <reason>
  Brief explanation of why you choose this route option
  </reason>
  <route>
  The name of the route to take
  </route>
  
Input: How to calculate C(5,3)?
Route selected: math
Reason: The given input is asking how to calculate the combination C(5,3), which is a mathematical concept. The "math" route is the best option to handle this question as it deals with mathematical operations and formulas.
Sending query: You are a professional math tutor. Solve the input math problem and show your work step by step. 
Input: How to calculate C(5,3)?


'To calculate the value of C(5,3), which represents the combination of 5 things taken 3 at a time, we can use the formula:\n\nC(n,r) = n! / (r! * (n-r)!)\n\nWhere:\n- n is the total number of items (in this case, 5)\n- r is the number of items being chosen (in this case, 3)\n- ! represents the factorial operation\n\nStep 1: Calculate the factorials.\n5! = 5 * 4 * 3 * 2 * 1 = 120\n3! = 3 * 2 * 1 = 6\n(5-3)! = (2)! = 2 * 1 = 2\n\nStep 2: Plug the values into the formula.\nC(5,3) = 5! / (3! * (5-3)!)\nC(5,3) = 120 / (6 * 2)\nC(5,3) = 120 / 12\nC(5,3) = 10\n\nTherefore, the value of C(5,3) is 10.'

In [ ]:
route("What is the origin of the word Valerie", routes)

Sending query: 
  Analyze the input and select the best route to take. 
  The available routes are:
  <routes>
    language: Handles language/linguistic related question
math: Handles math related question
  </routes>
  
  Your response must be in the following format:
  <reason>
  Brief explanation of why you choose this route option
  </reason>
  <route>
  The name of the route to take
  </route>
  
Input: What is the origin of the word Valerie
Route selected: language
Reason: The input question is asking about the origin of the word Valerie, which is a linguistic/language-related question. The "language" route is the best option to handle this type of question.
Sending query: You are a professional linguist. Answer the input use linguistic knowledge and give your explanations. 
Input: What is the origin of the word Valerie


'As a professional linguist, I can provide the following insights into the origin of the name Valerie:\n\nThe name Valerie is of Latin origin. It derives from the Latin name "Valeria," which was the feminine form of the masculine name "Valerius."\n\nThe name Valerius itself has its roots in the Latin word "valere," meaning "to be strong" or "to be well." This reflects the name\'s association with the concepts of strength, health, and well-being.\n\nThe name Valerie entered the English language through its French form, which became popular in the medieval period. In French, the name is spelled "Valérie" and is pronounced similarly to the English version.\n\nThe name Valerie has been used as a given name for women in various European cultures, including French, Italian, and Spanish-speaking regions. It has remained a popular choice for baby girls in many parts of the world, particularly in the Western world.\n\nIn summary, the name Valerie originates from the Latin name Valeria, which is

In [ ]:
route("What's weather in London", routes)

Sending query: 
  Analyze the input and select the best route to take. 
  The available routes are:
  <routes>
    language: Handles language/linguistic related question
math: Handles math related question
  </routes>
  
  Your response must be in the following format:
  <reason>
  Brief explanation of why you choose this route option
  </reason>
  <route>
  The name of the route to take
  </route>
  
Input: What's weather in London
Route selected: general information
Reason: The given input is about the weather in London, which is not a language/linguistic or math related question. It is a general query about the weather, which would be better suited for a "general information" or "weather" related route.


ValueError: Route general information not found

### Parallel

In [ ]:
from concurrent.futures import ThreadPoolExecutor

In [ ]:
def parallel(prompt: str, inputs: list[dict[str, str]], n_workers: int=3):
  executor = ThreadPoolExecutor(max_workers=n_workers)
  futures = [executor.submit(llm_call, prompt=prompt.format(**input)) for input in inputs]
  return [f.result() for f in futures]

In [ ]:
prompt = """
Translate the source text into {target_language}.
Your response should be in the following format:

<language>target language code</language>
<translation>tanslation in target language</translation>

Source text: {source_text}
"""

inputs = [
    {
        "target_language": "fr",
        "source_text": "Hello world"
    },
    {
        "target_language": "ja",
        "source_text": "Hello world"
    },
    {
        "target_language": "zh-CN",
        "source_text": "Hello world"
    }
]

parallel(prompt, inputs)

['<language>fr</language>\n<translation>Bonjour le monde</translation>',
 '<language>ja</language>\n<translation>こんにちは、世界</translation>',
 '<language>zh-CN</language>\n<translation>你好,世界</translation>']